# <b>Union, Intersect, Except.</b>

<ul>
<li>Para realizar operaciones entre conjuntos usando dos conjuntos de datos (data set), es importantes considerar que:</li>
<ul>
<li>Ambos data sets deben tener el mismo número de columnas.</li>
<li>El tipo de datos de cada columna de ambos data set debe ser el mismo.</li>
</ul>
</ul>

# <b>El operador UNION.</b>
<center><img src="https://drive.google.com/uc?id=1Pi4HxTEqvsVnl8QeBgYw4PWu8Q-RdvNe"></center>
<ul>
<li>Combina filas desde dos o más consultas eliminando registros duplicados: <br>
consulta1 <br>
<b>UNION [ALL]</b><br>
consulta2 <br>
<b>UNION [ALL]</b><br>
consulta3<br>
. . . . ;
</li>
<li> El operador <b>UNION ALL</b> no elimina los registros duplicados.</li>
<center><img src="https://drive.google.com/uc?id=1pv7fnoc6qImO18SzOY8TKlZUI3hJaCl2"></center>
<li>Los nombres de las columnas de la primera consulta se ocuparán en el data set resultante.</li>
<li>En caso de ser utilizadas, GROUP BY y HAVING son aplicadas sobre cada consulta y no sobre el resultado del operador UNION [ALL].</li>
<li>En el caso de ORDER BY, se aplicará sobre el data set resultante.</li>
<li> UNION vs JOIN:</li>
<center><img src="https://drive.google.com/uc?id=1YpBnLlGhLa7kR5HyOc6JE6GRWHHoZiZ1"></center>
</ul>

In [1]:
import sqlite3 
import pandas as pd

conn = sqlite3.connect("db//Chinook_Sqlite.sqlite") 
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #Seleccionamos la columna Nombre de la tabla sqlite_master, donde la columna type sea igual a 'table'
print(cursor.fetchall()) #Le pedimos a nuestro cursor que obtenga todos los resultados y los imprima:

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [3]:
sql_query="""SELECT FirstName, LastName, 'Employee' AS Type
               FROM employee
               UNION
               SELECT FirstName, LastName, 'Customer'
               FROM customer;"""
pd.read_sql(sql_query,conn)

,FirstName,LastName,Type
0,Aaron,Mitchell,Customer
1,Alexandre,Rocha,Customer
2,Andrew,Adams,Employee
3,Astrid,Gruber,Customer
4,Bjørn,Hansen,Customer
...,...,...,...
62,Steve,Murray,Customer
63,Terhi,Hämäläinen,Customer
64,Tim,Goyer,Customer
65,Victor,Stevens,Customer


#<b>El operador INTERSECT.</b>
<center><img src="https://drive.google.com/uc?id=1JMdqY3TT-YOb9AJbtCsthFmgTLB0okxa"></center>
<ul>
<li>Toma el resultado de dos consultas y retorna sin repetición los registros que son comunes a ambas:<br>
SELECT select_list1 <br>
FROM table1 <br>
<b>INTERSECT</b><br>
SELECT select_list2<br>
FROM table2<br>
</li>
</ul>

## Todos los clientes que tienen una factura:

In [4]:
sql_query_intersect="""SELECT CustomerId
                       FROM customer
                       INTERSECT
                       SELECT CustomerId
                       FROM invoice
                       ORDER BY CustomerId;"""
pd.read_sql(sql_query_intersect,conn)

,CustomerId
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


#<b>El operador EXCEPT.</b>
<center><img src="https://drive.google.com/uc?id=1nJrBjBodnXPPFjlWlHgd_P5dfr6_ySit"></center>
<ul>
<li>Compara el resultado de dos consultas y retorna los registros de la primera consulta que no son parte de los resultados de la segunda consulta.</li>
</ul>

## Los artistas que no tienen un álbum:

In [5]:
sql_query_except="""SELECT ArtistId
                    FROM artist
                    EXCEPT
                    SELECT ArtistId
                    FROM album;"""
pd.read_sql(sql_query_except,conn)

,ArtistId
0,25
1,26
2,28
3,29
4,30
...,...
66,192
67,193
68,194
69,195


In [6]:
sql_query_except="""explain query plan SELECT ArtistId
                    FROM artist
                    EXCEPT
                    SELECT ArtistId
                    FROM album;"""
pd.read_sql(sql_query_except,conn)

,id,parent,notused,detail
0,1,0,0,COMPOUND QUERY
1,2,1,0,LEFT-MOST SUBQUERY
2,5,2,0,SCAN TABLE artist USING COVERING INDEX IPK_Artist
3,11,1,0,EXCEPT USING TEMP B-TREE
4,13,11,0,SCAN TABLE album USING COVERING INDEX IFK_Albu...
